# Face Mask Detection

## Abstract
123

## Introduction
At the beginning of 2020, the COVID-19 epidemic has swept the world. In order to effectively prevent infection and continuous spread of the epidemic, various countries around the world have invested in a variety of AI vision anti-epidemic measures to urge people to protect themselves and others.

Face Mask Detection is one of the most important aspects.

This Face Mask Detection Project built with OpenCV, TensorFlow using Deep Learning and Maching Learning Classifier to detect face masks in images, videos and in real-time video streams.

## Dataset
This dataset consists of 3835 images belonging to two classes:

with_mask: 1916 images
without_mask: 1919 images

Source:
- [Real-World Masked Face Dataset，RMFD](https://github.com/X-zhangyang/Real-World-Masked-Face-Dataset)
- [Face-Mask-Detection](https://github.com/chandrikadeb7/Face-Mask-Detection/tree/master/dataset)

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
import cv2
import os

## Face Detection

In [32]:
# import dnn_model
net = cv2.dnn.readNetFromCaffe("deploy.prototxt", "res10_300x300_ssd_iter_140000_fp16.caffemodel")

def detect_img(net, image):
    blob = cv2.dnn.blobFromImage(image, 1.0, (300, 300), (104.0, 177.0, 123.0), False, False)
    net.setInput(blob)
    detections = net.forward()
    return detections

def crop_img(image, detections, dim):
    h, w, c = image.shape
    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.6:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            crop_img = image[startY:endY, startX:endX]
            crop_img = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)
            return crop_img
            
            
def show_detections(image, detections, dim, clf):
    h, w, c = image.shape
    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.6:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            
            crop_img = image[startY:endY, startX:endX]
            crop_img = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)
            resized = cv2.resize(crop_img, dim, interpolation = cv2.INTER_AREA)
            resized = resized.astype('float32')
#             plt.imshow(resized, cmap=plt.get_cmap('gray'))
            resized = resized.flatten()
            predict = clf.predict([resized])
            predict_proba = clf.predict_proba([resized])
            print(str(predict==1), predict_proba[0][predict])
            confidence = predict_proba[0][predict][0]
            
            text = str(predict==1) + "{:.2f}%".format(confidence * 100)
            y = startY - 10 if startY - 10 > 10 else startY + 10
            
            if predict[0]==1:
                cv2.rectangle(image, (startX, startY), (endX, endY),(0, 255, 0), 1)
                cv2.putText(image, text, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 255, 0), 2)
            else:
                cv2.rectangle(image, (startX, startY), (endX, endY),(0, 0, 255), 1)
                cv2.putText(image, text, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
    return image

In [50]:
# path to dataset
with_mask = "C:/Users/ZIJIE/Documents/Project/with_mask/"
without_mask = "C:/Users/ZIJIE/Documents/Project/without_mask/"
import joblib
clf = joblib.load('clf.model')
files = os.listdir(with_mask)
filename = random.choice(files)
file_path = with_mask + filename
width, height = 40, 40
dim = (width, height)

img = cv2.imread(file_path)
detections = detect_img(net, img)
img = show_detections(img, detections, dim, clf)
cv2.imshow("test", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

[False] [0.68502461]


## Load dataset

In [24]:
# path to dataset
with_mask = "C:/Users/ZIJIE/Documents/Project/with_mask/"
without_mask = "C:/Users/ZIJIE/Documents/Project/without_mask/"

import os
def load_face(path, width, height):
    data = []
    count = 0
    filenames = os.listdir(path)
    for file in filenames:
        file_path = path+file
        img = cv2.imread(file_path)
#         img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
        dim = (width, height)
        detections = detect_img(net, img)
        img = crop_img(img, detections, dim)
        if img is None:
            continue
        resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
        resized = resized.astype('float32')
        data.append(resized.flatten())
        count += 1
        if count % 100 == 0:
            print('loaded', count, 'faces')
    print('loaded', count, 'faces')
    return data

data_with_mask = load_face(with_mask, 40,40)
data_without_mask = load_face(without_mask, 40,40)
data = np.array(data_with_mask + data_without_mask)
label = np.array([1] * len(data_with_mask) + [0] * len(data_without_mask))

loaded 100 faces
loaded 200 faces
loaded 300 faces
loaded 400 faces
loaded 500 faces
loaded 600 faces
loaded 700 faces
loaded 800 faces
loaded 900 faces
loaded 1000 faces
loaded 1100 faces
loaded 1200 faces
loaded 1300 faces
loaded 1400 faces
loaded 1500 faces
loaded 1600 faces
loaded 1680 faces
loaded 100 faces
loaded 200 faces
loaded 300 faces
loaded 400 faces
loaded 500 faces
loaded 600 faces
loaded 700 faces
loaded 800 faces
loaded 900 faces
loaded 1000 faces
loaded 1100 faces
loaded 1200 faces
loaded 1300 faces
loaded 1400 faces
loaded 1500 faces
loaded 1600 faces
loaded 1700 faces
loaded 1800 faces
loaded 1900 faces
loaded 1916 faces


### Training

In [27]:
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV

def fit(X, y_label, n_components):
    pca = PCA(n_components=n_components, whiten=True)
    svc = SVC(kernel='rbf', class_weight='balanced', probability = True, verbose=True)
    model = make_pipeline(pca, svc)
    param_grid = {'svc__C': [1,5,10,50], 'svc__gamma':[0.0001, 0.0005, 0.001, 0.005]}
    grid = GridSearchCV(model, param_grid, verbose=True, n_jobs=-1)
    grid.fit(X, y_label)
    model = grid.best_estimator_
    return model

print('Training...')
n_components = 10
clf = fit(data, label, n_components)

import joblib
print("Saving model...")
joblib.dump(clf, 'clf.model')

Training...
Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    7.6s
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:   14.1s finished


[LibSVM]Saving model...


['clf.model']

### Cross-Validation

In [29]:
print('Cross-Validation...')
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
cv = ShuffleSplit(n_splits=10, test_size=0.3)
print(cross_val_score(clf, data, label, cv=cv, n_jobs=-1))

Cross-Validation...
[0.85912882 0.83873957 0.86005561 0.85912882 0.85820204 0.86005561
 0.86283596 0.84151993 0.85727525 0.84244671]


## Application
### Images

In [ ]:
import random
files = os.listdir(with_mask)
filename = random.choice(files)
file_path = with_mask + filename
width, height = 40, 40

img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
dim = (width, height)
resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
resized = resized.astype('float32')
plt.imshow(resized, cmap=plt.get_cmap('gray'))
img = resized.flatten()
clf.predict([img])

## Face Detection

In [ ]:
# import dnn_model
net = cv2.dnn.readNetFromCaffe("deploy.prototxt", "res10_300x300_ssd_iter_140000_fp16.caffemodel")

def detect_img(net, image):
    blob = cv2.dnn.blobFromImage(image, 1.0, (300, 300), (104.0, 177.0, 123.0), False, False)
    net.setInput(blob)
    detections = net.forward()
    return detections

def crop_img(image, detections, dim):
    h, w, c = image.shape
    img = []
    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.6:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            crop_img = image[startY:endY, startX:endX]
            crop_img = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)
#             img.append(crop_img)
            return crop_img
    return img
            
            
def show_detections(image, detections, dim, clf):
    h, w, c = image.shape
    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.6:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            
            crop_img = image[startY:endY, startX:endX]
            crop_img = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)
            resized = cv2.resize(crop_img, dim, interpolation = cv2.INTER_AREA)
            resized = resized.astype('float32')
#             plt.imshow(resized, cmap=plt.get_cmap('gray'))
            resized = resized.flatten()
            predict = clf.predict([resized])
            predict_proba = clf.predict_proba([resized])
            print(str(predict==1), predict_proba[0][predict])
            confidence = predict_proba[0][predict][0]
            
            text = str(predict==1) + "{:.2f}%".format(confidence * 100)
            y = startY - 10 if startY - 10 > 10 else startY + 10
            
            if predict[0]==1:
                cv2.rectangle(image, (startX, startY), (endX, endY),(0, 255, 0), 1)
                cv2.putText(image, text, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 255, 0), 2)
            else:
                cv2.rectangle(image, (startX, startY), (endX, endY),(255, 0, 0), 1)
                cv2.putText(image, text, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255, 0, 0), 2)
    return image

files = os.listdir(with_mask)
filename = random.choice(files)
file_path = with_mask + filename
width, height = 40, 40
dim = (width, height)

img = cv2.imread(file_path)
detections = detect_img(net, img)
img = show_detections(img, detections, dim, clf)
plt.imshow(img, cmap=plt.get_cmap('gray'))